# Activity 9

This activity tests your skills
in approaching an engineering problem
as solving a system of nonlinear algebraic equations.

## Scenario

Referring to [the diagram](./act-09.png),
a device has an active power consumption $P$
when operating at a lagging power factor $\Phi$.
The requisite power is conveyed by the
resistance $R$,
inductance $L$
and the capacitor $C$,
from a source
configured to provide a steady RMS voltage $U$
at a steady frequency $F$.

You are to solve for the following quantities.
- $V$, the RMS operating voltage of the device
- $\delta_{\text{l}}$,
  the phase angle of the voltage phasor at which the device receives power
- $\delta_{\text{s}}$
  the phase angle of the voltage phasor provided by the source

## Modelling

Cast the original task as
solving a system of nonlinear algebraic equations
in three unknowns
(namely, $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$).
Introduce a vector $\boldsymbol{x}$ 
to collect the unknowns in the order presented above,
so that the system of equations can be written in vector form as
$\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$.
Set the voltage phasor across the input terminals of the load
to have the reference phase angle.

## Core code

In [1]:
import math as mt

import numpy as np
import scipy as sp

Define a Python function
`f()` that takes

- one positional argument,
  `x`, representing $\boldsymbol{x}$;
  and
- four keyword arguments,
  `U`, `F`, `R`, `L`, `C`, `P`, and `Phi`
  representing $U$, $F$, $R$, $L$, $C$, $P$, and $\Phi$ respectively.

`f()` returns a NumPy array representing
$\boldsymbol{f}\!\left(\boldsymbol{x}\right)$.

Moreover, define a Python function `dfdx()`
that takes the same arguments as `f()`,
and returns a NumPy array representing
the Jacobian of $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ w.r.t. $\boldsymbol{x}$.


In [2]:
def f(x, U, F, R, L, C, P, Phi):
    # Variables
    V = x[0]
    delta_l = x[1]
    delta_s = x[2]
    
    # Circuit Parameters
    omega = 2 * mt.pi * F
    X_L = omega * L
    B_C = omega * C
    
    # Load Reactive Power (Lagging PF implies Q > 0)
    # phi is the power factor angle
    phi = np.arccos(Phi)
    Q_load = P * np.tan(phi)
    
    # Intermediate terms
    theta = delta_l - delta_s
    Z_sq = R**2 + X_L**2
    
    # Pre-calculate common trigonometric and voltage terms
    term_VU = V * U
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    # Real Power Balance Equation (derived from Nodal Analysis)
    P_calc = (1.0 / Z_sq) * (R * (term_VU * cos_theta - V**2) - X_L * (term_VU * sin_theta))
    
    # Reactive Power Balance Equation
    Q_from_source = (1.0 / Z_sq) * (X_L * (term_VU * cos_theta - V**2) + R * (term_VU * sin_theta))
    Q_cap = B_C * V**2
    Q_calc = Q_from_source + Q_cap
    
    # The system of equations f(x) = 0
    y = np.zeros(3)
    y[0] = P_calc - P
    y[1] = Q_calc - Q_load
    y[2] = delta_l - 0.0  # Constraint: Load voltage phasor is the reference (0 degrees)
    
    return y

def dfdx(x, U, F, R, L, C, P, Phi):
    # Variables
    V = x[0]
    delta_l = x[1]
    delta_s = x[2]
    
    # Circuit Parameters
    omega = 2 * mt.pi * F
    X_L = omega * L
    B_C = omega * C
    
    # Intermediate terms
    theta = delta_l - delta_s
    Z_sq = R**2 + X_L**2
    K = 1.0 / Z_sq
    
    term_VU = V * U
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    # Derivatives of P_calc
    # dP/dV
    dPdV = K * (R * (U * cos_theta - 2*V) - X_L * U * sin_theta)
    # dP/dtheta (where theta = delta_l - delta_s)
    dPdtheta = K * (R * (-term_VU * sin_theta) - X_L * (term_VU * cos_theta))
    
    # Derivatives of Q_calc
    # dQ/dV
    dQdV = K * (X_L * (U * cos_theta - 2*V) + R * U * sin_theta) + 2 * B_C * V
    # dQ/dtheta
    dQdtheta = K * (X_L * (-term_VU * sin_theta) + R * (term_VU * cos_theta))
    
    # Jacobian Matrix
    J = np.zeros((3, 3))
    
    # Row 0: d(P_calc - P) / dx
    J[0, 0] = dPdV              # w.r.t V
    J[0, 1] = dPdtheta * 1      # w.r.t delta_l (dtheta/ddelta_l = 1)
    J[0, 2] = dPdtheta * (-1)   # w.r.t delta_s (dtheta/ddelta_s = -1)
    
    # Row 1: d(Q_calc - Q_load) / dx
    J[1, 0] = dQdV
    J[1, 1] = dQdtheta * 1
    J[1, 2] = dQdtheta * (-1)
    
    # Row 2: d(delta_l - 0) / dx
    J[2, 0] = 0
    J[2, 1] = 1
    J[2, 2] = 0
    
    return J

## Assessment

Every time you solve for $\boldsymbol{x}$,
- define Python variables
  `U`, `F`, `R`, `L`, `C`, `P`, and `PHI`
  for storing the values of parameters
  $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$,
  respectively;
- define a Python variable `x_ini`
  for storing your initial guesses of the variables;
- use [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html),
  particularly its `args` keyword argument;
- define a Python variable `x_out` to store $\boldsymbol{x}$;
- verify if $\boldsymbol{f}\!\left(\boldsymbol{x}\right) = \boldsymbol{0}$ is sufficiently satisfied;
- print the value of $V$ in kilovolts;
  and
- print the values of $\delta_{\text{l}}$ and $\delta_{\text{s}}$ in degrees.

### Task 1

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
40 kilowatts, 0.87.

In [3]:
U = 13.8e3
F = 60
R = 0.21
L = 5.570e-3
C = 1.560e-3
P = 40e3
PHI = 0.87

# Initial Guess: V=U, angles=0
x_ini = np.array([U, 0.0, 0.0])

# Solve
sol = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx)
x_out = sol.x

# Output
V = x_out[0]
delta_l = x_out[1]
delta_s = x_out[2]

# Verify f(x) is close to 0
residuals = f(x_out, U, F, R, L, C, P, PHI)
print(f"Residuals: {residuals}")
print(f"Success: {sol.success}")

# Print required values
print(f"V: {V/1000:.4f} kilovolts")
print(f"delta_l: {np.degrees(delta_l):.4f} degrees")
print(f"delta_s: {np.degrees(delta_s):.4f} degrees")

Residuals: [-3.56521923e-10  1.02227204e-09  2.53637226e-26]
Success: True
V: 0.0070 kilovolts
delta_l: 0.0000 degrees
delta_s: 54.7553 degrees


### Task 2

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
13.8 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
32 kilowatts, 0.97.

In [4]:
U = 13.8e3
F = 50
R = 0.17
L = 6.685e-3
C = 1.872e-3
P = 32e3
PHI = 0.97

x_ini = np.array([U, 0.0, 0.0])

sol = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx)
x_out = sol.x

V = x_out[0]
delta_l = x_out[1]
delta_s = x_out[2]

print(f"Residuals: {f(x_out, U, F, R, L, C, P, PHI)}")
print(f"V: {V/1000:.4f} kilovolts")
print(f"delta_l: {np.degrees(delta_l):.4f} degrees")
print(f"delta_s: {np.degrees(delta_s):.4f} degrees")

Residuals: [-2.11584847e-08  3.30251169e-07 -6.24407840e-25]
V: 0.0050 kilovolts
delta_l: -0.0000 degrees
delta_s: 71.3077 degrees


### Task 3

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
214 kilowatts, 0.87.

In [5]:
U = 32.5e3
F = 60
R = 0.21
L = 5.570e-3
C = 1.560e-3
P = 214e3
PHI = 0.87

x_ini = np.array([U, 0.0, 0.0])

sol = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx)
x_out = sol.x

V = x_out[0]
delta_l = x_out[1]
delta_s = x_out[2]

print(f"Residuals: {f(x_out, U, F, R, L, C, P, PHI)}")
print(f"V: {V/1000:.4f} kilovolts")
print(f"delta_l: {np.degrees(delta_l):.4f} degrees")
print(f"delta_s: {np.degrees(delta_s):.4f} degrees")

Residuals: [-1.86264515e-09  5.41331246e-09  2.98027877e-26]
V: 0.0160 kilovolts
delta_l: 0.0000 degrees
delta_s: 54.7550 degrees


### Task 4

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
173 kilowatts, 0.97.

In [6]:
U = 32.5e3
F = 50
R = 0.17
L = 6.685e-3
C = 1.872e-3
P = 173e3
PHI = 0.97

x_ini = np.array([U, 0.0, 0.0])

sol = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx)
x_out = sol.x

V = x_out[0]
delta_l = x_out[1]
delta_s = x_out[2]

print(f"Residuals: {f(x_out, U, F, R, L, C, P, PHI)}")
print(f"V: {V/1000:.4f} kilovolts")
print(f"delta_l: {np.degrees(delta_l):.4f} degrees")
print(f"delta_s: {np.degrees(delta_s):.4f} degrees")

Residuals: [-2.03726813e-10  3.20142135e-09  5.21525488e-26]
V: 0.0116 kilovolts
delta_l: 0.0000 degrees
delta_s: 71.3075 degrees


### Task 5

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
69.0 kilovolts,
60 hertz,
0.21 ohms, 5.570 millihenrys, 1.560 millifarads,
421 kilowatts, 0.87.

In [7]:
U = 69.0e3
F = 60
R = 0.21
L = 5.570e-3
C = 1.560e-3
P = 421e3
PHI = 0.87

x_ini = np.array([U, 0.0, 0.0])

sol = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx)
x_out = sol.x

V = x_out[0]
delta_l = x_out[1]
delta_s = x_out[2]

print(f"Residuals: {f(x_out, U, F, R, L, C, P, PHI)}")
print(f"V: {V/1000:.4f} kilovolts")
print(f"delta_l: {np.degrees(delta_l):.4f} degrees")
print(f"delta_s: {np.degrees(delta_s):.4f} degrees")

Residuals: [ 1.08120730e-06 -6.30796421e-07 -2.83885050e-25]
V: 0.0148 kilovolts
delta_l: -0.0000 degrees
delta_s: 54.7508 degrees


### Task 6

Determine $V$, $\delta_{\text{l}}$, and $\delta_{\text{s}}$
when $U$, $F$, $R$, $L$, $C$, $P$, and $\phi$
have respective values of
32.5 kilovolts,
50 hertz,
0.17 ohms, 6.685 milllihenrys, 1.872 millifarads,
317 kilowatts, 0.97.

In [8]:
U = 32.5e3
F = 50
R = 0.17
L = 6.685e-3
C = 1.872e-3
P = 317e3
PHI = 0.97

x_ini = np.array([U, 0.0, 0.0])

sol = sp.optimize.root(f, x_ini, args=(U, F, R, L, C, P, PHI), jac=dfdx)
x_out = sol.x

V = x_out[0]
delta_l = x_out[1]
delta_s = x_out[2]

print(f"Residuals: {f(x_out, U, F, R, L, C, P, PHI)}")
print(f"V: {V/1000:.4f} kilovolts")
print(f"delta_l: {np.degrees(delta_l):.4f} degrees")
print(f"delta_s: {np.degrees(delta_s):.4f} degrees")

Residuals: [-3.90573405e-08  1.80589268e-07  6.05525871e-26]
V: 0.0212 kilovolts
delta_l: 0.0000 degrees
delta_s: 71.3118 degrees


## Instructions

Do not use any library or module other than those pre-imported.

### Scoring

In each Task,
- a correct use of
  [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)
  merits two (2) points;
- obtaining a correct $V$ merits two (3) points;
- obtaining a correct $\delta_{\text{l}}$ merits two (3) points;
- obtaining a correct $\delta_{\text{s}}$ merits two (3) points;
  and
- obtaining $\boldsymbol{f}\!\left(\boldsymbol{x}\right)$ close enough to $\boldsymbol{0}$ merits six (9) points.


Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 120 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-09_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-09_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-12-11*